In [184]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from functions import *
from scipy.stats import stats, norm
from fancyimpute import KNN
from sklearn import preprocessing
from sklearn.feature_extraction import FeatureHasher
import joblib

# 1. Data Processing

In [185]:
pd.set_option('display.max_columns', None)
df1 = pd.read_csv('ml_case_test_data.csv')
df2 = pd.read_csv('ml_case_test_hist_data.csv')

In [186]:
df2.drop(['price_date', 'price_p1_var'],inplace=True,axis=1)
df2 = df2.groupby(['id'], as_index=False).mean()
df = df1.merge(df2, how='inner', on='id')
df.head() 

,id,activity_new,campaign_disc_ele,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_first_activ,date_modif_prod,date_renewal,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix
0,ed18c9981ce183ef347246cdcc55f743,bwpaswkpcilmlklklcapcwwumwaodaoo,NaN,foosdfpfkusacimwkcsosbicdxkicaua,42132,0,3215,2011-09-06,2016-09-06,NaN,2015-08-27,2015-09-07,NaN,NaN,NaN,NaN,4001.08,3215,0.0,130.43,0.114603,0.098261,40.606701,f,296.90,9.63,9.63,1,488.25,4,lxidpiddsbxsbosboudacockeimpuepw,20.000,0.103568,0.073708,40.728885,24.437330,16.291555
1,9c341e73685fcff6e3527ebd51da3fe5,NaN,NaN,foosdfpfkusacimwkcsosbicdxkicaua,3537,0,563,2010-12-13,2016-12-31,NaN,2010-12-13,2016-01-01,NaN,NaN,NaN,NaN,518.22,563,0.0,23.20,0.142174,0.000000,44.444710,f,80.66,0.12,0.12,1,29.18,5,lxidpiddsbxsbosboudacockeimpuepw,15.000,0.000000,0.000000,44.444710,0.000000,0.000000
2,97dd64b33716aadc4e22c4ab85d1c7a4,NaN,NaN,foosdfpfkusacimwkcsosbicdxkicaua,6543,0,399,2010-09-04,2016-09-04,NaN,2015-08-27,2015-09-05,NaN,NaN,NaN,NaN,639.65,399,0.0,130.32,0.115182,0.098841,40.606701,f,37.91,15.53,15.53,1,74.59,5,lxidpiddsbxsbosboudacockeimpuepw,17.321,0.104431,0.074728,40.606699,24.364017,16.242678
3,cf81de72ff7997ed10729751059cf7a3,apdekpcbwosbxepsfxclislboipuxpop,NaN,foosdfpfkusacimwkcsosbicdxkicaua,186838,0,12091,2011-01-03,2017-01-03,NaN,2011-01-03,2016-01-04,NaN,NaN,NaN,NaN,2634.23,738,0.0,16.42,0.164058,0.086163,44.311378,f,93.12,27.00,27.00,1,210.18,5,lxidpiddsbxsbosboudacockeimpuepw,11.951,0.086593,0.000000,44.266930,0.000000,0.000000
4,c7c8ee9daf2a864d159d45a2373ada72,NaN,NaN,usilxuppasemubllopkaafesmlibmsdf,19051,0,0,2011-10-07,2016-10-07,NaN,2015-08-27,2015-10-09,NaN,NaN,NaN,NaN,2801.47,0,0.0,0.00,0.141434,0.000000,44.311378,f,0.00,8.62,8.62,1,223.23,4,lxidpiddsbxsbosboudacockeimpuepw,13.150,0.000000,0.000000,44.326190,0.000000,0.000000


In [187]:
# check duplicated rows
quant_dup(df['id'])

0

In [188]:
# Replace values
value = 'No'
df[['activity_new', 'campaign_disc_ele', 'channel_sales']] = df[['activity_new', 'campaign_disc_ele', 'channel_sales']].fillna(value=value)

# Split by year, month, day
df[['year_activ','month_activ','day_activ']] = df.date_activ.str.split("-", expand=True) 
df[['year_end','month_end','day_end']] = df.date_end.str.split("-", expand=True)
df[['year_first_activ','month_first_activ','day_first_activ']] =  df.date_first_activ.str.split("-", expand=True) 
df[['year_modif_prod','month_modif_prod','day_modif_prod']] =  df.date_modif_prod.str.split("-", expand=True) 
df[['year_renewal','month_renewal','day_renewal']] =  df.date_renewal.str.split("-", expand=True) 

# Conver to numerical values
df[['year_activ','month_activ','day_activ','year_end','month_end','day_end','year_first_activ','month_first_activ','day_first_activ','year_modif_prod','month_modif_prod','day_modif_prod','year_renewal','month_renewal','day_renewal']] = df[['year_activ','month_activ','day_activ','year_end','month_end','day_end','year_first_activ','month_first_activ','day_first_activ','year_modif_prod','month_modif_prod','day_modif_prod','year_renewal','month_renewal','day_renewal']].apply(pd.to_numeric, errors='coerce')

# Change the scale to days
df['date_activ_days'] = round((df['year_activ']*365) + (df['month_activ']*30) + df['day_activ'])
df['date_end_days'] = round((df['year_end']*365) + (df['month_end']*30) + df['day_end'])
df['date_first_activ_days'] = round((df['year_first_activ']*365) + (df['month_first_activ']*30) + df['day_first_activ'])
df['date_modif_prod_days'] = round((df['year_modif_prod']*365) + (df['month_modif_prod']*30) + df['day_modif_prod'])
df['date_renewal_days'] = round((df['year_renewal']*365) + (df['month_renewal']*30) + df['day_renewal'])

# Calculate the duration from active date
df['duration'] = df['date_end_days'] - df['date_activ_days']
df['duration_first_activ'] = df['date_first_activ_days'] - df['date_activ_days']
df['duration_modif_prod'] = df['date_modif_prod_days'] - df['date_activ_days']
df['duration_renewal'] = df['date_renewal_days'] - df['date_activ_days']

# Finally drop all the date columns except durations
df.drop(['date_activ', 'date_end', 'date_first_activ', 'date_modif_prod','date_renewal','date_activ_days', 'date_end_days','date_first_activ_days', 
         'date_modif_prod_days','date_renewal_days','year_activ', 'month_activ', 'day_activ','year_end', 'month_end', 'day_end','year_first_activ',
         'month_first_activ','day_first_activ','year_modif_prod','month_modif_prod','day_modif_prod','year_renewal','month_renewal','day_renewal'],axis=1, inplace=True)
df.head()

,id,activity_new,campaign_disc_ele,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix,duration,duration_first_activ,duration_modif_prod,duration_renewal
0,ed18c9981ce183ef347246cdcc55f743,bwpaswkpcilmlklklcapcwwumwaodaoo,No,foosdfpfkusacimwkcsosbicdxkicaua,42132,0,3215,NaN,NaN,NaN,NaN,4001.08,3215,0.0,130.43,0.114603,0.098261,40.606701,f,296.90,9.63,9.63,1,488.25,4,lxidpiddsbxsbosboudacockeimpuepw,20.000,0.103568,0.073708,40.728885,24.437330,16.291555,1825,NaN,1451.0,1461.0
1,9c341e73685fcff6e3527ebd51da3fe5,No,No,foosdfpfkusacimwkcsosbicdxkicaua,3537,0,563,NaN,NaN,NaN,NaN,518.22,563,0.0,23.20,0.142174,0.000000,44.444710,f,80.66,0.12,0.12,1,29.18,5,lxidpiddsbxsbosboudacockeimpuepw,15.000,0.000000,0.000000,44.444710,0.000000,0.000000,2208,NaN,0.0,1848.0
2,97dd64b33716aadc4e22c4ab85d1c7a4,No,No,foosdfpfkusacimwkcsosbicdxkicaua,6543,0,399,NaN,NaN,NaN,NaN,639.65,399,0.0,130.32,0.115182,0.098841,40.606701,f,37.91,15.53,15.53,1,74.59,5,lxidpiddsbxsbosboudacockeimpuepw,17.321,0.104431,0.074728,40.606699,24.364017,16.242678,2190,NaN,1818.0,1826.0
3,cf81de72ff7997ed10729751059cf7a3,apdekpcbwosbxepsfxclislboipuxpop,No,foosdfpfkusacimwkcsosbicdxkicaua,186838,0,12091,NaN,NaN,NaN,NaN,2634.23,738,0.0,16.42,0.164058,0.086163,44.311378,f,93.12,27.00,27.00,1,210.18,5,lxidpiddsbxsbosboudacockeimpuepw,11.951,0.086593,0.000000,44.266930,0.000000,0.000000,2190,NaN,0.0,1826.0
4,c7c8ee9daf2a864d159d45a2373ada72,No,No,usilxuppasemubllopkaafesmlibmsdf,19051,0,0,NaN,NaN,NaN,NaN,2801.47,0,0.0,0.00,0.141434,0.000000,44.311378,f,0.00,8.62,8.62,1,223.23,4,lxidpiddsbxsbosboudacockeimpuepw,13.150,0.000000,0.000000,44.326190,0.000000,0.000000,1825,NaN,1420.0,1462.0


In [189]:
# Drop columns with 40% NaN
df = df.dropna(thresh = df.shape[0]*.4, axis=1)

In [190]:
# Impute with KNN
num_cols = ['cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_12m',
       'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_p1',
       'forecast_price_energy_p2', 'forecast_price_pow_p1', 'imp_cons',
       'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'pow_max',
       'price_p2_var', 'price_p3_var', 'price_p1_fix', 'price_p2_fix',
       'price_p3_fix','duration','duration_modif_prod','duration_renewal']

knn_imputer = KNN()
df[num_cols] = KNN(k=5).fit_transform(df[num_cols])

Imputing row 1/4024 with 0 missing, elapsed time: 4.769
Imputing row 101/4024 with 0 missing, elapsed time: 4.770
Imputing row 201/4024 with 0 missing, elapsed time: 4.773
Imputing row 301/4024 with 0 missing, elapsed time: 4.775
Imputing row 401/4024 with 0 missing, elapsed time: 4.776
Imputing row 501/4024 with 0 missing, elapsed time: 4.778
Imputing row 601/4024 with 0 missing, elapsed time: 4.779
Imputing row 701/4024 with 0 missing, elapsed time: 4.781
Imputing row 801/4024 with 0 missing, elapsed time: 4.782
Imputing row 901/4024 with 0 missing, elapsed time: 4.784
Imputing row 1001/4024 with 0 missing, elapsed time: 4.786
Imputing row 1101/4024 with 0 missing, elapsed time: 4.788
Imputing row 1201/4024 with 0 missing, elapsed time: 4.790
Imputing row 1301/4024 with 0 missing, elapsed time: 4.791
Imputing row 1401/4024 with 0 missing, elapsed time: 4.793
Imputing row 1501/4024 with 0 missing, elapsed time: 4.795
Imputing row 1601/4024 with 0 missing, elapsed time: 4.797
Imputing 

In [191]:
df[num_cols].isna().sum()

cons_12m                    0
cons_gas_12m                0
cons_last_month             0
forecast_cons_12m           0
forecast_cons_year          0
forecast_discount_energy    0
forecast_meter_rent_12m     0
forecast_price_energy_p1    0
forecast_price_energy_p2    0
forecast_price_pow_p1       0
imp_cons                    0
margin_gross_pow_ele        0
margin_net_pow_ele          0
nb_prod_act                 0
net_margin                  0
num_years_antig             0
pow_max                     0
price_p2_var                0
price_p3_var                0
price_p1_fix                0
price_p2_fix                0
price_p3_fix                0
duration                    0
duration_modif_prod         0
duration_renewal            0
dtype: int64

In [192]:
df[num_cols].to_csv('num_cols_imputed_test.csv', index=False)
df[num_cols] = pd.read_csv('num_cols_imputed_test.csv')

In [193]:
num_cols_imputed = df[num_cols]
# Remove outliers
print("Before", df.shape)
normalized_num_cols = preprocessing.normalize(num_cols_imputed)
z = np.abs(stats.zscore(normalized_num_cols))
df = df[(compare_nan_array(np.less, z, 3)).all(axis=1)].copy()
print("After", df.shape)

Before (4024, 31)
After (3292, 31)


In [194]:
id = df[['id']]

In [195]:
fh = FeatureHasher(input_type='string')
f = df[['activity_new']]
h = fh.transform(f)

h=h.toarray()
h=h.transpose()

h=pd.DataFrame(h, columns=['activity_new'])
df.drop(['id','activity_new'], axis=1, inplace=True)

df.reset_index()
h.reset_index()
df.merge(h,how='inner', left_index=True, right_index=True)

,campaign_disc_ele,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix,duration,duration_modif_prod,duration_renewal,activity_new
0,No,foosdfpfkusacimwkcsosbicdxkicaua,42132.0,0.0,3215.0,4001.08,3215.0,0.0,130.43,0.114603,0.098261,40.606701,f,296.90,9.63,9.63,1.0,488.25,4.0,lxidpiddsbxsbosboudacockeimpuepw,20.000,0.103568,0.073708,40.728885,24.437330,16.291555,1825.0,1451.0,1461.0,0.0
1,No,foosdfpfkusacimwkcsosbicdxkicaua,3537.0,0.0,563.0,518.22,563.0,0.0,23.20,0.142174,0.000000,44.444710,f,80.66,0.12,0.12,1.0,29.18,5.0,lxidpiddsbxsbosboudacockeimpuepw,15.000,0.000000,0.000000,44.444710,0.000000,0.000000,2208.0,0.0,1848.0,0.0
2,No,foosdfpfkusacimwkcsosbicdxkicaua,6543.0,0.0,399.0,639.65,399.0,0.0,130.32,0.115182,0.098841,40.606701,f,37.91,15.53,15.53,1.0,74.59,5.0,lxidpiddsbxsbosboudacockeimpuepw,17.321,0.104431,0.074728,40.606699,24.364017,16.242678,2190.0,1818.0,1826.0,0.0
3,No,foosdfpfkusacimwkcsosbicdxkicaua,186838.0,0.0,12091.0,2634.23,738.0,0.0,16.42,0.164058,0.086163,44.311378,f,93.12,27.00,27.00,1.0,210.18,5.0,lxidpiddsbxsbosboudacockeimpuepw,11.951,0.086593,0.000000,44.266930,0.000000,0.000000,2190.0,0.0,1826.0,0.0
4,No,usilxuppasemubllopkaafesmlibmsdf,19051.0,0.0,0.0,2801.47,0.0,0.0,0.00,0.141434,0.000000,44.311378,f,0.00,8.62,8.62,1.0,223.23,4.0,lxidpiddsbxsbosboudacockeimpuepw,13.150,0.000000,0.000000,44.326190,0.000000,0.000000,1825.0,1420.0,1462.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4016,No,foosdfpfkusacimwkcsosbicdxkicaua,5208.0,6560.0,0.0,764.28,0.0,0.0,16.44,0.142881,0.000000,45.806878,t,0.00,17.13,17.13,2.0,60.92,3.0,lxidpiddsbxsbosboudacockeimpuepw,10.392,0.000000,0.000000,44.620467,0.000000,0.000000,1485.0,1066.0,1121.0,0.0
4018,No,lmkebamcaaclubfxadlmueccxoimlema,5485.0,0.0,0.0,597.26,0.0,0.0,16.09,0.167086,0.088444,45.806878,f,0.00,18.08,18.08,1.0,47.24,6.0,kamkkxfxxuwbdslkwifmmcsiusiuosws,10.350,0.087686,0.000000,44.546560,0.000000,0.000000,2555.0,0.0,2191.0,0.0
4019,No,foosdfpfkusacimwkcsosbicdxkicaua,329908.0,0.0,-21618.0,31706.64,-21618.0,0.0,92.08,0.110083,0.093746,40.606701,f,-2614.57,-14.38,-14.38,1.0,1832.66,4.0,NaN,99.450,0.100698,0.070995,40.606701,24.364016,16.242676,1825.0,0.0,1461.0,0.0
4020,No,foosdfpfkusacimwkcsosbicdxkicaua,82518.0,0.0,5782.0,8213.88,5782.0,0.0,129.60,0.114658,0.099545,40.606701,f,559.71,-0.96,-0.96,1.0,938.27,4.0,lxidpiddsbxsbosboudacockeimpuepw,24.249,0.103206,0.072975,40.565971,24.339579,16.226385,1825.0,1059.0,1463.0,0.0


In [202]:
df = pd.get_dummies(df, drop_first=True)
df['channel_sales_sddiedcslfslkckwlfkdpoeeailfpeds'],df['origin_up_usapbepcfoloekilkwsdiboslwaxobdp'] = 0,0

# 2. Prediction with the Trained Model

In [197]:
best_grid_gb = joblib.load('gb.pkl')
y_pred_gb = best_grid_gb.predict(df)
y_pred_prob_gb = best_grid_gb.predict_proba(df)[:, 1]

In [198]:
y_pred_prob_gb_df = pd.DataFrame(y_pred_prob_gb, columns=['churn probability'])
result = pd.concat([id, y_pred_prob_gb_df], names=['ID','Churn Probability'], axis=1)
result.head()

,id,churn probability
0,ed18c9981ce183ef347246cdcc55f743,0.107594
1,9c341e73685fcff6e3527ebd51da3fe5,0.160484
2,97dd64b33716aadc4e22c4ab85d1c7a4,0.093115
3,cf81de72ff7997ed10729751059cf7a3,0.071251
4,c7c8ee9daf2a864d159d45a2373ada72,0.059820


In [201]:
result.to_csv("result.csv",index=False)